# Day 4 - Lab 1: Automated Testing & Quality Assurance

**Objective:** Generate a comprehensive `pytest` test suite for the database-connected FastAPI application, including tests for happy paths, edge cases, and tests that use advanced fixtures for database isolation.

**Estimated Time:** 135 minutes

**Introduction:**
Welcome to Day 4! An application without tests is an application that is broken by design. Today, we focus on quality assurance. You will act as a QA Engineer, using an AI co-pilot to build a robust test suite for the API you created yesterday. This is a critical step to ensure our application is reliable and ready for production.

For definitions of key terms used in this lab, please refer to the [GLOSSARY.md](../../GLOSSARY.md).

## Step 1: Setup

We will load the source code for our main application from `app/main.py`. Providing the full code as context is essential for the LLM to generate accurate and relevant tests.

**Model Selection:**
For generating tests, models with strong code understanding and logical reasoning are best. `gpt-4.1`, `o3`, `codex-mini`, and `gemini-2.5-pro` are all excellent choices for this task.

**Helper Functions Used:**
- `setup_llm_client()`: To configure the API client.
- `get_completion()`: To send prompts to the LLM.
- `load_artifact()`: To read our application's source code.
- `save_artifact()`: To save the generated test files.
- `clean_llm_output()`: To clean up the generated Python code.

In [7]:
import sys
import os

# Add the project's root directory to the Python path to ensure 'utils' can be imported.
# The project root should be 220372-AG-AISOFTDEV-Team-3-PromptPioneers
current_dir = os.getcwd()
project_root = os.path.abspath(os.path.join(current_dir, '..'))

print(f"Current directory: {current_dir}")
print(f"Project root: {project_root}")

if project_root not in sys.path:
    sys.path.insert(0, project_root)

# Also add current directory
if current_dir not in sys.path:
    sys.path.insert(0, current_dir)

from utils import setup_llm_client, get_completion, save_artifact, clean_llm_output

client, model_name, api_provider = setup_llm_client(model_name="gpt-4.1")

# Load the application code from the root folder main.py to provide context for test generation
main_py_path = os.path.join(project_root, "main.py")
try:
    with open(main_py_path, 'r', encoding='utf-8') as f:
        app_code = f.read()
    print(f"Successfully loaded main.py from {main_py_path}")
except FileNotFoundError:
    app_code = None
    print(f"Warning: Could not find main.py at {main_py_path}. Lab may not function correctly.")
except Exception as e:
    app_code = None
    print(f"Warning: Error loading main.py: {e}. Lab may not function correctly.")

Current directory: c:\Users\labadmin\Desktop\220372-AG-AISOFTDEV-Team-3-PromptPioneers\LABS
Project root: c:\Users\labadmin\Desktop\220372-AG-AISOFTDEV-Team-3-PromptPioneers


2025-11-06 11:57:25,602 ag_aisoftdev.utils INFO LLM Client configured provider=openai model=gpt-4.1 latency_ms=None artifacts_path=None


Successfully loaded main.py from c:\Users\labadmin\Desktop\220372-AG-AISOFTDEV-Team-3-PromptPioneers\main.py


## Step 2: The Challenges

### Challenge 1 (Foundational): Generating "Happy Path" Tests

**Task:** Generate basic `pytest` tests for the ideal or "happy path" scenarios of your CRUD endpoints.

**Instructions:**
1.  Create a prompt that asks the LLM to act as a QA Engineer.
2.  Provide the `app_code` as context.
3.  Instruct the LLM to generate a `pytest` test function for the `POST /users/` endpoint, asserting that a user is created successfully (e.g., checking for a `201 Created` or `200 OK` status code and verifying the response body).
4.  Generate another test for the `GET /users/` endpoint.
5.  Save the generated tests into a file named `tests/test_main_simple.py`.

**Expected Quality:** A Python script containing valid `pytest` functions that test the basic, successful operation of your API.

In [9]:
# TODO: Write a prompt to generate happy path tests for your API.
happy_path_tests_prompt = f"""
You are an expert QA Engineer tasked with creating comprehensive pytest tests for a FastAPI application. 

Please analyze the following FastAPI application code and generate happy path tests that verify the successful operation of the main CRUD endpoints:

APPLICATION CODE:
{app_code}

Requirements for the tests:
1. Act as a professional QA Engineer
2. Generate pytest test functions for the following happy path scenarios:
   - POST /file-attachments/ - Test successful creation of a file attachment (assert 201 status and verify response data)
   - GET /file-attachments/ - Test successful retrieval of all file attachments (assert 200 status and verify response structure)
   - GET /file-attachments/{{id}} - Test successful retrieval of a specific file attachment (assert 200 status and verify response data)
   - PUT /file-attachments/{{id}} - Test successful full update of a file attachment (assert 200 status and verify updated data)
   - PATCH /file-attachments/{{id}} - Test successful partial update of a file attachment (assert 200 status and verify updated fields)

3. Include necessary imports (pytest, FastAPI TestClient, etc.)
4. Use FastAPI's TestClient for testing
5. Create realistic test data that matches the Pydantic models
6. Include proper assertions for status codes and response content
7. Follow pytest naming conventions (test_*)
8. Add descriptive docstrings for each test function
9. Make tests independent and self-contained

Generate clean, production-ready pytest code that thoroughly tests the happy path scenarios for this file attachment API.
"""

print("--- Generating Happy Path Tests ---")
if app_code:
    generated_happy_path_tests = get_completion(happy_path_tests_prompt, client, model_name, api_provider)
    cleaned_tests = clean_llm_output(generated_happy_path_tests, language='python')
    print(cleaned_tests)
    save_artifact(cleaned_tests, "tests/test_happy_paths.py", overwrite=True)
else:
    print("Skipping test generation because app code is missing.")

--- Generating Happy Path Tests ---
"""
Pytest Test Suite for File Attachment CRUD Endpoints

This suite contains happy path tests for the main CRUD operations
of the AI-Powered Requirement Analyzer's file attachment API.

Requirements:
- Tests are independent and self-contained.
- Each test uses realistic data and validates status codes & response content.
"""

import pytest
from fastapi.testclient import TestClient
from datetime import datetime

from main import app

client = TestClient(app)

# --------------------------------------------------------------------------
# Helper: Default payload for file attachments
# --------------------------------------------------------------------------

def get_file_attachment_payload(
    conversation_id=1,
    original_filename="requirements.docx",
    file_type="docx",
    file_size=12345,
    storage_path="/storage/reqs/requirements.docx",
    extracted_text="Extracted requirement text.",
    status="uploaded",
    additional_metadata='{"auth

### Challenge 2 (Intermediate): Generating Edge Case Tests

**Task:** Prompt the LLM to generate tests for common edge cases, such as providing invalid data or requesting a non-existent resource.

**Instructions:**
1.  Create a new prompt.
2.  Provide the `app_code` as context.
3.  Instruct the LLM to write two new test functions:
    * A test for the `POST /users/` endpoint that tries to create a user with an email that already exists, asserting that the API returns a `400 Bad Request` error.
    * A test for the `GET /users/{user_id}` endpoint that requests a non-existent user ID, asserting that the API returns a `404 Not Found` error.

**Expected Quality:** Two new `pytest` functions that verify the application handles common error scenarios correctly.

In [ ]:
# TODO: Write a prompt to generate edge case tests.
edge_case_tests_prompt = f"""
You are an expert QA Engineer tasked with creating comprehensive edge case tests for a FastAPI application. 

Please analyze the following FastAPI application code and generate edge case tests that verify the application handles common error scenarios correctly:

APPLICATION CODE:
{app_code}

Requirements for the edge case tests:
1. Act as a professional QA Engineer focused on breaking the application
2. Generate 5-8 pytest test functions covering the following edge case scenarios:
   
   **Invalid Data Scenarios:**
   - POST /file-attachments/ with invalid file_size (negative, zero, exceeding 50MB limit)
   - POST /file-attachments/ with invalid conversation_id (negative, zero)
   - POST /file-attachments/ with invalid file_type (unsupported format)
   - POST /file-attachments/ with missing required fields
   - POST /file-attachments/ with empty/invalid original_filename
   
   **Non-existent Resource Scenarios:**
   - GET /file-attachments/{{non_existent_id}} (assert 404 Not Found)
   - PUT /file-attachments/{{non_existent_id}} (assert 404 Not Found)
   - PATCH /file-attachments/{{non_existent_id}} (assert 404 Not Found)
   - DELETE /file-attachments/{{non_existent_id}} (assert 404 Not Found)
   
   **Invalid ID Scenarios:**
   - GET/PUT/PATCH/DELETE with negative file_id (assert 400 Bad Request)
   - GET/PUT/PATCH/DELETE with zero file_id (assert 400 Bad Request)
   
   **Empty Update Scenarios:**
   - PATCH /file-attachments/{{id}} with empty request body (assert 400 Bad Request)

3. Include necessary imports (pytest, FastAPI TestClient, etc.)
4. Use FastAPI's TestClient for testing
5. Include proper assertions for status codes and error messages
6. Follow pytest naming conventions (test_*)
7. Add descriptive docstrings for each test function explaining the edge case
8. Make tests independent and self-contained
9. Test both validation errors (422) and business logic errors (400/404)
10. Verify error response format and meaningful error messages

Generate clean, production-ready pytest code that thoroughly tests edge cases and error handling for this file attachment API. Focus on scenarios that could break the application or cause security issues.
"""

print("--- Generating Edge Case Tests ---")
if app_code:
    generated_edge_case_tests = get_completion(edge_case_tests_prompt, client, model_name, api_provider)
    cleaned_edge_case_tests = clean_llm_output(generated_edge_case_tests, language='python')
    print(cleaned_edge_case_tests)
    save_artifact(cleaned_edge_case_tests, "tests/test_edge_cases.py", overwrite=True)
else:
    print("Skipping test generation because app code is missing.")

### Challenge 3 (Advanced): Testing with an Isolated Database Fixture

**Task:** Generate a `pytest` fixture that creates a fresh, isolated, in-memory database for each test session. Then, refactor your tests to use this fixture. This is a critical pattern for professional-grade testing.

> **Hint:** Why use an isolated database? Running tests against your actual development database can lead to data corruption and flaky, unreliable tests. A pytest fixture that creates a fresh, in-memory database for each test ensures that your tests are independent, repeatable, and have no side effects.

**Instructions:**
1.  Create a prompt that asks the LLM to generate a `pytest` fixture.
2.  This fixture should configure a temporary, in-memory SQLite database using SQLAlchemy.
3.  It needs to create all the database tables before the test runs and tear them down afterward.
4.  Crucially, it must override the `get_db` dependency in your FastAPI app to use this temporary database during tests.
5.  Save the generated fixture code to a special file named `tests/conftest.py`.
6.  Finally, create a new test file, `tests/test_main_with_fixture.py`, and ask the LLM to rewrite the happy-path tests from Challenge 1 to use the new database fixture.

**Expected Quality:** Two new files, `tests/conftest.py` and `tests/test_main_with_fixture.py`, containing a professional `pytest` fixture for database isolation and tests that are correctly refactored to use it.

In [ ]:
# TODO: Write a prompt to generate the pytest fixture for an isolated test database.
db_fixture_prompt = f"""
# Your prompt here
"""

print("--- Generating Pytest DB Fixture ---")
if app_code:
    generated_db_fixture = get_completion(db_fixture_prompt, client, model_name, api_provider)
    cleaned_fixture = clean_llm_output(generated_db_fixture, language='python')
    print(cleaned_fixture)
    save_artifact(cleaned_fixture, "tests/conftest.py")
else:
    print("Skipping fixture generation because app context is missing.")

# TODO: Write a prompt to refactor the happy path tests to use the new fixture.
refactor_tests_prompt = f"""
# Your prompt here
"""

print("\n--- Generating Refactored Tests ---")
if app_code:
    refactored_tests = get_completion(refactor_tests_prompt, client, model_name, api_provider)
    cleaned_refactored_tests = clean_llm_output(refactored_tests, language='python')
    print(cleaned_refactored_tests)
    save_artifact(cleaned_refactored_tests, "tests/test_main_with_fixture.py")
else:
    print("Skipping test refactoring because app context is missing.")

## Lab Conclusion

Fantastic work! You have built a comprehensive test suite for your API, moving from simple happy path tests to advanced, isolated database testing. You've learned how to use AI to brainstorm edge cases and generate complex fixtures. Having a strong test suite like this gives you the confidence to make changes to your application without fear of breaking existing functionality.

> **Key Takeaway:** Using AI to generate tests is a massive force multiplier for quality assurance. It excels at creating boilerplate test code, brainstorming edge cases, and generating complex setup fixtures, allowing developers to build more reliable software faster.